# Practical session 4: Modulating internal states and Sensing other robot's attributes

In the last practical session, we saw how to run multiple behaviors in parallel on multiple robots. In this section we will see more advanced methods for combining behaviors by modulating their activations according to internal states of the robot and by allowing them to sense the attributes of others. In order to start with a clean basis, let's first provide the definition of several behaviors. These four behaviors are simply implementations of the [Braitenberg vehicles](https://docs.google.com/presentation/d/1FlAUyvNynYU4mDBE2o20pf2Bn5PC_9Id3SK8sCMfr-Q/edit#slide=id.g31e1b425a3_0_0) we have seen in class, where the sources (i.e. what is sensed by the proximeters) are other epucks:

In [5]:
def fear(robot):
    left, right = robot.prox_activations(tracked_objects=["ePuck"])
    left_wheel = left
    right_wheel = right
    return left_wheel, right_wheel

def aggression(robot):
    left, right = robot.prox_activations(tracked_objects=["ePuck"])
    left_wheel = right
    right_wheel = left
    return left_wheel, right_wheel

def love_cuddly(robot):
    left, right = robot.prox_activations(tracked_objects=["ePuck"])
    left_wheel = 1 - left
    right_wheel = 1 - right   
    return left_wheel, right_wheel

def love_shy(robot):
    left, right = robot.prox_activations(tracked_objects=["ePuck"])
    left_wheel = 1 - right
    right_wheel = 1 - left   
    return left_wheel, right_wheel

Note that the arguments of the behavior functions above are now named `robot`, whereas they were named `epuck`in the previous sessions. This is actually strictly equivalent, the argument name being an arbitrary name. We choose here to call it `robot` in order to avoid a confusion when dealing with multiple epucks. The only important thing is that you use the same name in the function body (i.e `robot` here).

We define a fifth behavior for obstacle avoidance, where obstacles are walls, pilars, trees and cups:

In [6]:
def obstacle_avoidance(robot):
    left, right = robot.prox_activations(tracked_objects=["20cm", "Tree", "Cup"])
    left_wheel = 1 - right
    right_wheel = 1 - left   
    return left_wheel, right_wheel

Open V-REP and load the scene `epuck-scene-4.ttt` located in the directory `robotics/pyvrep_epuck/vrep-scenes`.

**Reminder:** First import the functions `open_session` and `close_session`. Then, obtain a reference to the simulator and the E-Puck by calling `open_session`. Since there are three epucks in the scene, we call it like this:

In [2]:
from simulator_interface import open_session, close_session
simulator, epuck1, epuck2, epuck3 = open_session(n_epucks=3)

Then execute the code you will encounter in the notebook or the code you will write for answering the questions. Whenever you want to restart from scratch (e.g. because something goes wrong or because you want to restart from scratch), first close the session by executing:

In [ ]:
close_session(simulator)

This will properly close all the running processes. Then restart the notebook (`Kernel -> Restart`) and open the session again:

In [3]:
from simulator_interface import open_session, close_session
simulator, epuck1, epuck2, epuck3 = open_session(n_epucks=3)

And you will restart with a clean session.

## Emergent behaviors

When running one or several behaviors on a robot, you might observe behavioral responses that are not present in any of the behavior definitions. This usually doesn't mean that there is a bug, it could just be the result of an emergent behavioral property. Let's analyze this on a quick example, by running the `obstacle_avoidance` behavior on each epuck:

In [5]:
for e in simulator.robots:
    e.attach_behavior(obstacle_avoidance, freq=10)
    e.start_all_behaviors()

Behavior obstacle_avoidance started
Behavior obstacle_avoidance started
Behavior obstacle_avoidance started


Note that the cell above is a shortcut to execute the same set of instructions (the indented lines) on all the robots returned by the `open_session` function (the one you use to connect to the simulator). It can become very useful when you want to run the same set of instructions on all the robots, as it is the case here. 

**Q1:** You will observe that the ePucks seem to be attracted to objects that are not handled by the behavior (e.g. other ePucks). Analyse this phenomena and explain below why it is occuring in a few lines:

*Double click on this cell to enter your answer*

## Analyzing robot interactions

Let's analyze the interaction between two robots that run different behaviors. First close the current session and open a new one with only two epucks that will be controlled (the third epuck is still part of the scene but we don't need it here):

In [3]:
close_session(simulator, epuck1, epuck2, epuck3)
simulator, epuck1, epuck2 = open_session(n_epucks=2)

**Q2:** In addition to the `obstacle_avoidance` behavior, run several combinations of the four other behaviors defined above. For example, run `obstacle_avoidance` and `fear` on `epuck1` ; and `obstacle_avoidance` and `aggression` on `epuck2`. Find two of these combinations that you consider as interesting (e.g. because they result in a relatively complex interaction pattern, or because they can be linked to animal behavior) and describe them in a few lines:

In [34]:
# Your code here


*Your answer here*

## Weighting behaviors

As we have seen in the previous session, it is possible to run several behaviors in parallel on the same robot. When doing it, the motor activation sent to each wheel corresponds to the average of the motor activation returned by each behavior (this averaging is implemented internally, you don't need to worry about it). 

It is also possible to specify the weight of each running behavior, i.e. how much it will count in the averaging. This is done by returning three values in the function defining a behavior (instead of two as we were doing until now: one for the left wheel activation and one for the right one). For example, if we want to run the `obstacle_avoidance` behavior with a weight of 1 and the `fear` behavior with a weight of 0.5, we write:

In [8]:
# First detach all behaviors on both robots:
for e in simulator.robots:
    e.detach_all_behaviors()
    e.stop()
    
# define the obstacle_avoidance behavior with a weight of 1. This is indicated by the third value returned by the function:
def obstacle_avoidance(robot):
    left, right = robot.prox_activations(tracked_objects=["20cm", "Tree", "Cup"])
    # The weight associated with this behavior is the third returned value, here 1:
    return 1 - right, 1 - left , 1.

# define the fear behavior with a weight of 0.5. This is indicated by the third value returned by the function:
def fear(robot):
    left, right = robot.prox_activations(tracked_objects=["ePuck"])
    # The weight associate with this behavior is the third return value, here 0.5:
    return left, right, 0.5

# Attach and start both behaviors on both robots:
for e in simulator.robots:
    e.attach_behavior(obstacle_avoidance, freq=10)
    e.attach_behavior(fear, freq=10)
    e.start_all_behaviors()

Behavior fear started
Behavior obstacle_avoidance started
Behavior fear started
Behavior obstacle_avoidance started


By doing this, the wheel activations returned by the `obstacle_avoidance` behavior will have twice more weight than those returned by the `fear` behavior. For example, if `obstacle_avoidance` returns 0.6 for the left wheel, and `fear` returns 0.9, then the total activation of that wheel will be $(0.6 * 1 + 0.9 * 0.5) / (1 + 0.5) = 0.7$ (i.e. the average of both values weighted by their respective activation). Note that when no weight is provided in a behavior definition (i.e. when the behavior function returns only two values as usual), the corresponding behavior is set with a default weight of one.

## Weighting behaviors according to internal states

This weighting is particularly useful to activate a behavior according to some internal states of the robot. Let's consider a robot that eats spheres (as in the previous session) and that eating those spheres allows to raise a simulated energy level of the robot. We want to continuously compute the energy level of the robot according to how much spheres it has recently eaten. To do so, we first need a way to know when a robot eats a sphere. This is done by executing:

In [ ]:
epuck1.has_eaten()

The cell above return `True` if the robot (`epuck1` here) has eaten a sphere since the last call of the function. Run both the `obstacle avoidance` and the `foraging` behavior on an ePuck and execute the `has_eaten` function above at different times to understand correctly how it works. Remember we have already defined a `foraging_behavior` in the last session, which can be implemented like this:

In [9]:
def foraging(robot):
    left, right = robot.prox_activations(tracked_objects=["Sphere"])
    left_activation = right
    right_activation = left
    return left_activation, right_activation

In [ ]:
# First start sphere apparition in the environment:
simulator.start_sphere_apparition(period=10)

# Detach all existing behaviors, and attach then start obstacle_avoidance and foraging on all robots:
for e in simulator.robots:
    e.detach_all_behaviors()
    
    e.attach_behavior(obstacle_avoidance, freq=10)
    e.attach_behavior(foraging, freq=10)
    e.start_all_behaviors()
    


Now we want to continuously compute the energy level of a robot, so that the level decreases slowly when nothing is eaten and increases whenever food is consumed. This can be done by attaching and starting a *routine* to a robot. The definition of a routine is very similar to the definition of a behavior, except that it doesn't return any value (whereas a behavior always returns the left and right wheel activations, and optionally a weight). Thus, a routine corresponds to a set of instructions that are executed at a particular frequency (as in a behavior), e.g. to compute some robot's internal states according to its interaction with the environment.

Let's define a routine called `foraging_drive` that computes the energy level as specified above:

In [14]:
def foraging_drive(robot): 
    if robot.has_eaten():
        robot.energy_level += 0.2  # if the robot has eaten a sphere, increase its energy level by 0.2
    else:
        robot.energy_level -= 0.01  # otherwise (nothing eaten), decrease the energy level by 0.01
    # The line below bounds the value of the energy level between 0 and 1
    robot.energy_level = min(1., max(robot.energy_level, 0.))

As for a behavior, the function defining a routine takes a `robot` as an argument representing the ePuck on which the routine is attached. Attaching a routine is done with: 

In [15]:
epuck1.attach_routine(foraging_drive, freq=1)

As for a behavior, a routine is attached to an epuck at a given frequency. However, contrary to behaviors, routines are not directly involved in motor control and therefore can be run at a lower frequency (1Hz above). The code of the `foraging_drive` function above will be executed each second (1Hz frequency), consequently modulating the ePuck energy level stored in `robot.energy_level` according to how good the robot is at catching spheres.

Before starting a routine, we need to define an initial level of energy. Let's do it on `epuck1`:

In [17]:
epuck1.energy_level = 0.5  # Note that the energy level is bounded between 0 and 1 in the foraging_drive function

This is mandatory because `energy_level` is incremented or decremented in the `foraging_drive` definition. If not initialized, it will throw an error.

Let's now effectively start this routine:

In [18]:
epuck1.start_routine(foraging_drive)

Routine foraging_drive started


And check the evolution of the energy level in `epuck_1`:

In [ ]:
print(epuck1.energy_level)

Let the robot behave in the environment and re-execute the cell above to track the evolution of the energy level.

**Q3:** Modify the `foraging` behavior so that it is weighted according to the energy level of the robot: the lower the energy level, the higher `foraging` is weighted. However, as seen in Q1 above, the ePuck might still be attracted to spheres even when the `foraging` weight is null. Solve this issue by adding another behavior that drives the robot away from sphere according to the energy level: the higher the energy level, the more the robot is repulsed from spheres.

## Sensing other robot's attributes

It can be useful to allow a robot to sense attributes from other robots. For example, one might want to define different species of robot (cats and mouses for example) so that how a robot interact with another depends on their respective species. To achieve this, it is **not** recommended to modify the ePuck labels directly in V-REP (as a general rule, never modify the object labels in the VREP scene). Instead, each robot can be set with a specific attribute, e.g `epuck.species = "cat"` and that attribute can be sensed by other robots whenever it is detected by a proximeter. Let's try it with three epucks:

In [27]:
close_session(simulator)
simulator, epuck1, epuck2, epuck3 = open_session(n_epucks=3)

where we define a `species` attribute for each robot:

In [28]:
epuck1.species = "cat"
epuck2.species = "mouse"
epuck3.species = "mouse"

So we have one cat and two mouses. Place the robots so that `epuck3` (mouse) is sensing `epuck1` (cat) on its left proximeter and `epuck2` (mouse) on its right proximeter. One can access attributes of sensed epucks by first executing:

In [29]:
(left, right), (epuck_left, epuck_right) = epuck3.prox_activations(tracked_objects=["ePuck"], return_epucks=True)

This cell above calls the `prox_activations` function of `epuck3` with an additional argument, `return_epucks`, which is set to `True`. When called like this, `prox_activations` will return the left and right proximeter activations as usual, as well as the references to the epucks that are sensed by the proximeters, if any. Then we can access a specific attribute of those ePucks by executing:

In [ ]:
left_species, right_species = epuck3.sensed_epuck_attributes(epuck_left, epuck_right, "species", default_value="none")

print(left_species)
print(right_species)

The `sensed_epuck_attributes` function takes 4 arguments: the epuck sensed by the left proximeter (called `epuck_left` in the example above), the one sensed by the right proximeter (called `epuck_right` above), the name of the attribute we are interested in (here `"species"`) and the default value to return if a proximeter is not sensing any ePuck (here `none`).

Move the robots in the scene and check how the attribute sensing is changing accordingly by re-executing the two above cells. Then try with other attributes, for example by setting a `age` attribute to the robots and sensing it from the proximeters.

Let's now extend the `fear` behavior so that mouses are only afraid by the cat but not by the other mouse:

In [31]:
def fear(robot):
    # call prox_activation with return_epucks set to True:
    (left, right), (epuck_left, epuck_right) = robot.prox_activations(tracked_objects=["ePuck"], return_epucks=True)
    # Retrieve the species attributes of the sensed epucks:
    left_species, right_species = robot.sensed_epuck_attributes(epuck_left, epuck_right, "species", default_value ="none")
    # Set the left wheel activation to the value of the left sensor if a cat is on the left. Otherwise set it to 0:
    left_activation = left if left_species == "cat" else 0
    # Set the right wheel activation to the value of the right sensor if a cat is on the right. Otherwise set it to 0:
    right_activation = right if right_species == "cat" else 0
    # Return both wheel activation as usual:
    return left_activation, right_activation

And run this behavior together with the `obstacle_avoidance` one on `epuck3`:

In [32]:
epuck3.behavior_mixer.set_mode("average")
epuck3.detach_all_behaviors()
epuck3.attach_behavior(obstacle_avoidance, freq=10)
epuck3.attach_behavior(fear, freq=10)
epuck3.start_all_behaviors()

Behavior obstacle_avoidance started
Behavior fear started


Check in the simulator that `epuck3` (mouse) is now avoiding `epuck1` (cat) but not `epuck2` (mouse). We can modify the `species` attribute of robots on the fly. Let's swap the species of `epuck1` and `epuck2`:

In [33]:
epuck1.species = "mouse"
epuck2.species = "cat"

Check in the simulator that `epuck3` (mouse) is now avoiding `epuck2` (cat) but not `epuck1` (mouse).

**Q4:** Use the new functionalities we have seen in this session to design the following system:
Two epucks are equipped with behaviors to avoid obstacle and catch spheres, as well as being attracted or repulsed by the other epuck. Attraction and repulsion depend on the energy level of the other robot: the higher this level the more attraction, the lower this level the more repulsion. 

In [9]:
for e in simulator.robots:
    e.detach_all_behaviors()

In [ ]:
# Your code here


**Q5**: Modify the previous simulation such that attraction and repulsion depend on how much a robot has been close to others in the recent past. To do so, define a `social_drive` routine that modulates the value of a `social_need` attribute in each robot. This social need continuously increases when other robots are far and decreases when a robot comes closer to its conspecifics.

## About the mini-project

The aim of the mini project is to show that you have understood all the concepts we have seen during these three practical sessions and that you can integrate them in a single demo. You can of course modify the V-REP scene at your will, e.g adding cups or trees. Don't forget to save you modified V-REP scene (`File -> Save scene as` in V-REP). You can start your project in a new Jupyter Notebook (`File -> New notebook -> Python3`) in the menu bar at the top of this notebook). Use both text cells (explaining what you are doing) and code cells (with the corresponding code to execute).

Therefore, your project will consists of two files: a V-REP scene (extension `.ttt`) a Jupyter Notebook (extension `.ipynb`). Don't forget to save them, e.g. on a USB stick, before logging out from the UPF computers. See [this slide](https://docs.google.com/presentation/d/1FlAUyvNynYU4mDBE2o20pf2Bn5PC_9Id3SK8sCMfr-Q/edit#slide=id.g34900c6ead_0_0) for more information on the mini-project.